# Neural Network - Subscribe Prediction

1. Train two models:
   - **Baseline Logistic Regression** (simple, interpretable)
   - **Neural Network (MLP)** (more complex, flexible)
2. Compare performance using clear metrics and visualizations.
3. Highlight **what leaders should focus on** in evaluation and decision‑making.

In [ ]:
import pandas as pd
from numpy.random import default_rng
import matplotlib.pyplot as plt, warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss, roc_curve, precision_recall_curve
from sklearn.calibration import calibration_curve
from scipy.io import arff

rng = default_rng(0)
warnings.filterwarnings("ignore")

## Dataset Description
- Author: Paulo Cortez, Sérgio Moro 
- Source: UCI 
- Cite: S. Moro, R. Laureano and P. Cortez. Using Data Mining for Bank Direct Marketing: An Application of the CRISP-DM Methodology. In P. Novais et al. (Eds.), Proceedings of the European Simulation and Modelling Conference - ESM'2011, pp. 117-121, Guimarães, Portugal, October, 2011. EUROSIS.

### Background
The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be (or not) subscribed.

The classification goal is to predict if the client will subscribe a term deposit (variable y).

### Features:
**bank client data:**
- age (numeric)
- job : type of job (categorical: "admin.","unknown","unemployed","management","housemaid","entrepreneur", "student","blue-collar","self-employed","retired","technician","services")
- marital : marital status (categorical: "married","divorced","single"; note: "divorced" means divorced or widowed)
- education (categorical: "unknown","secondary","primary","tertiary")
- default: has credit in default? (binary: "yes","no")
- balance: average yearly balance, in euros (numeric)
- housing: has housing loan? (binary: "yes","no")
- loan: has personal loan? (binary: "yes","no")

**related with the last contact of the current campaign:**
- contact: contact communication type (categorical: "unknown","telephone","cellular")
- day: last contact day of the month (numeric)
- month: last contact month of year (categorical: "jan", "feb", "mar", ..., "nov", "dec")
- duration: last contact duration, in seconds (numeric)

**other attributes:**
- campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
- pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric, -1 means client was not previously contacted)
- previous: number of contacts performed before this campaign and for this client (numeric)
- poutcome: outcome of the previous marketing campaign (categorical: "unknown","other","failure","success")

**output variable (desired target):**
- y - has the client subscribed a term deposit? (binary: "yes","no")

In [ ]:
# Load Dataset
!wget -q https://raw.githubusercontent.com/Jihun-ust/ust-mail-557/main/NeuralNetwork/phpkIxskf.arff
file_name = "phpkIxskf.arff"
data, meta = arff.loadarff(file_name)
df = pd.DataFrame(data)
# Rename columns
col_map = {
    "V1": "age",
    "V2": "job",
    "V3": "marital",
    "V4": "education",
    "V5": "default",
    "V6": "balance",
    "V7": "housing",
    "V8": "loan",
    "V9": "contact",
    "V10": "day",
    "V11": "month",
    "V12": "duration",
    "V13": "campaign",
    "V14": "pdays",
    "V15": "previous",
    "V16": "poutcome",
    "Class": "subscribed"
}

df = df.rename(columns=col_map)

# Convert data types
def decode_bytes(val):
    if isinstance(val, (bytes, bytearray)):
        try:
            return val.decode('utf-8')  # decode to string
        except:
            return str(val)  # fallback to str
    if isinstance(val, str) and val.startswith("b'"):  # string that looks like b'...'
        return val.strip("b'").strip("'")
    return val

df = df.applymap(decode_bytes)

# Ensure numeric columns are numeric
num_cols = ["age","balance","day","duration","campaign","pdays","previous"]
df[num_cols] = df[num_cols].apply(pd.to_numeric, errors='coerce')

# Convert target (binary)
df["subscribed"] = df["subscribed"].replace({"1": 1, "2": 0})

print(df.head())

## Train/Test split
Hold out 20% of data for final evaluation.

In [ ]:
X = df.drop(columns=['subscribed'])
y = df['subscribed']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

num_cols = ['age','balance','day', 'duration', 'campaign', 'pdays', 'previous']
cat_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']

pre = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

## Baseline - Logistic Regression
Simple, interpretable, runs fast.

In [ ]:
logreg = Pipeline([
    ('pre', pre),
    ('clf', LogisticRegression(max_iter=3000))
])
logreg.fit(X_train, y_train)
probs_lr = logreg.predict_proba(X_test)[:,1]

auc_lr = roc_auc_score(y_test, probs_lr)
pr_lr = average_precision_score(y_test, probs_lr)
brier_lr = brier_score_loss(y_test, probs_lr)

result = pd.DataFrame({
    "LR result":{
        "roc_auc_score": auc_lr,
        "average_precision_score": pr_lr,
        "brier_score_loss": brier_lr
    }
})

display(result.round(3))

## Neural Network (MLP)
A simple feedforward network (multi‑layer perceptron). More flexible but less interpretable.

In [ ]:
mlp = Pipeline([
    ('pre', pre),
    ('clf', MLPClassifier(hidden_layer_sizes=(16, 8)))
])
mlp.fit(X_train, y_train)
probs_nn = mlp.predict_proba(X_test)[:,1]

auc_nn = roc_auc_score(y_test, probs_nn)
pr_nn = average_precision_score(y_test, probs_nn)
brier_nn = brier_score_loss(y_test, probs_nn)

result = pd.DataFrame({
    "NN result":{
        "roc_auc_score": auc_nn,
        "average_precision_score": pr_nn,
        "brier_score_loss": brier_nn
    }
})

display(result.round(3))

# Training Loss curve
clf = mlp.named_steps["clf"]
plt.figure(figsize=(6,4))
plt.plot(clf.loss_curve_)
plt.xlabel("epoch"); plt.ylabel("loss")
plt.title("MLP training loss curve")
plt.tight_layout(); plt.show()

## Baseline vs. Neural Net
- **ROC‑AUC** (ranking quality): How well the model ranks likely buyers above non-buyers. (higher=better)
- **PR‑AUC** (precision‑recall for rare positives): How well the model finds true buyers among the top predictions when buyers are rare. (higher=better)
- **Brier score** (calibration): How well the predicted probabilities match reality (calibration). (lower=better)

In [ ]:
# ROC curves
fpr_lr, tpr_lr, _ = roc_curve(y_test, probs_lr)
fpr_nn, tpr_nn, _ = roc_curve(y_test, probs_nn)

plt.figure(figsize=(6,5))
plt.plot(fpr_lr, tpr_lr, label=f"Logistic (AUC={auc_lr:.3f})")
plt.plot(fpr_nn, tpr_nn, label=f"Neural Net (AUC={auc_nn:.3f})")
plt.plot([0,1],[0,1],'--', color="gray")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve Comparison")
plt.legend()
plt.show()

# Precision-Recall curves
prec_lr, rec_lr, _ = precision_recall_curve(y_test, probs_lr)
prec_nn, rec_nn, _ = precision_recall_curve(y_test, probs_nn)

plt.figure(figsize=(6,5))
plt.plot(rec_lr, prec_lr, label=f"Logistic (PR-AUC={pr_lr:.3f})")
plt.plot(rec_nn, prec_nn, label=f"Neural Net (PR-AUC={pr_nn:.3f})")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve Comparison")
plt.legend()
plt.show()

# Calibration (reliability) curve
prob_true_lr, prob_pred_lr = calibration_curve(y_test, probs_lr, n_bins=12, strategy='quantile')
prob_true_nn, prob_pred_nn = calibration_curve(y_test, probs_nn, n_bins=12, strategy='quantile')

plt.figure(figsize=(6,5))
plt.plot(prob_pred_lr, prob_true_lr, marker='o', label=f"Logistic (Brier={brier_lr:.3f})")
plt.plot(prob_pred_nn, prob_true_nn, marker='o', label=f"Neural Net (Brier={brier_nn:.3f})")
plt.plot([0,1],[0,1],'--')
plt.xlabel("Predicted probability (bin mean)")
plt.ylabel("Observed frequency")
plt.title("Calibration")
plt.legend()
plt.show()

results = pd.DataFrame({
    "Logistic Regression": {
        "ROC-AUC (higher=better)": auc_lr,
        "PR-AUC (higher=better)": pr_lr,
        "Brier (lower=better)": brier_lr
    },
    "Neural Net": {
        "ROC-AUC (higher=better)": auc_nn,
        "PR-AUC (higher=better)": pr_nn,
        "Brier (lower=better)": brier_nn
    }
})

display(results.T.round(3))

## Decision Making
- **If baseline is close**: stick with logistic regression (simpler, transparent, easier to monitor).
- **If neural net shows clear lift**: consider adopting, but demand **extra checks**:
  - Calibration: are probabilities reliable?
  - Fairness: does performance differ by device/country?
  - Ops: can it run fast enough for homepage personalization?
- **Always tie back to business metrics**: incremental conversions, ROI, and fairness guardrails.


- The lesson: More complex is not always better. Leaders should weigh **performance vs. simplicity, interpretability, fairness, and operations**.